In [ ]:
import cv2
import numpy as np
import time
import math

WIND_X = 1000
WIND_Y = 800

axes = {"x" : 0, "y" : 1, "z" : 2}

markup_color = (250, 250, 250)

class Vector:
    def __init__ (self, coords_ = []):
        self.coords = coords_
    
    def get_coords (self):
        return self.coords
    
    def dotproduct (self, v):
        return sum ((a * b) for a, b in zip (self.coords, v.coords))

    def length (self):
        return math.sqrt (self.dotproduct (self))

    def subtr (self, v):
        return Vector ([a - b for a, b in zip (self.coords, v.coords)])

    def add (self, v):
        return Vector ([a + b for a, b in zip (self.coords, v.coords)])

    def mul (self, coeff):
        return Vector ([a * coeff for a in self.coords])

    def cos (self, v):
        return self.dotproduct (v) / (self.length () * v.length ())
    
    def change_coord (self, coord, val = 0, increment = False, invert = False):
        if (invert == True):
            self.coords [axes [coord]] *= -1
        
        else:
            if (increment == True):
                self.coords [axes [coord]] += val

            else:
                self.coords [axes [coord]] = val
    
    def rotate_2d (self, axis1, axis2, angle):
        orig1 = self.coords [axes [axis1]]
        orig2 = self.coords [axes [axis2]]
        
        rot1 =   orig1 * math.cos (angle) + orig2 * math.sin (angle)
        rot2 = - orig1 * math.sin (angle) + orig2 * math.cos (angle)

        self.coords [axes [axis1]] = rot1
        self.coords [axes [axis2]] = rot2
        
    def copy (self):
        return Vector (self.coords)

class Object:
    def __init__ (self, coords_):
        self.coords = coords_
        
    def draw (self, canvas):
        pass
    
    def get_coords (self):
        return self.coords.get_coords ()
    
    def get_coords_vec (self):
        return self.coords

class Light_emitter (Object):
    def __init__ (self, coords_, color_ = (255, 255, 255)):
        Object.__init__ (self, coords_)
        
        self.color = color_
    
    def get_color (self):
        return self.color
    
    def draw (self, canvas):
        canvas.draw_3d_circle (self.coords, 10, markup_color)

class Surface (Object):
    def __init__ (self, coords_):
        Object.__init__ (self, coords_)
    
    def draw (self, canvas):
        self.render (canvas)
    
    def render (self, canvas):
        pass
    
    def iterate_elements (self):
        pass
    
    def calc_element_lightening (self):
        pass
    
    def _norm_3_points (self, p1, p2, p3):
        u = p2.subtr (p1).get_coords ()
        v = p3.subtr (p1).get_coords ()
        
        n = Vector ([u [1] * v [2] - u [2] * v [1],
                     u [2] * v [0] - u [0] * v [2],
                     u [0] * v [1] - u [1] * v [0]])
        
        return n

class Triangle(Surface):
    def __init__ (self, p1_, p2_, p3_, color_ = (100, 100, 255)):
        Object.__init__ (self, p1_)
        
        self.p1 = p1_
        self.p2 = p2_
        self.p3 = p3_
        
        self.color = color_
    
    def draw (self, canvas, emitter, shift = [0, 0, 0]):
        self.render (canvas, emitter, shift)
    
    def render (self, canvas, emitter, shift = Vector ([0, 0, 0])):
        n = self._norm_3_points (self.p1, self.p2, self.p3)
        
        tr_color = self.calc_lightening (n, emitter)
                
        canvas.draw_3d_triangle (self.p1.add (shift),
                                 self.p2.add (shift),
                                 self.p3.add (shift),
                                 tr_color)
    
    def calc_lightening (self, n, emitter):
        light_vec = emitter.coords.subtr (self.coords)
        
        coeff = max (n.cos (light_vec), 0)
        
        result = [int (channel * coeff) for channel in self.color]
        
        return result
    
    def get_color (self):
        return self.color

class Triangle_mesh (Surface):
    def __init__ (self, coords_, effective_spatial_dimension_ = 0):
        Surface.__init__ (self, coords_)
        
        self.triangles = []
        
        self.effective_spatial_dimension = effective_spatial_dimension_
        
    def generate_triangulation (self):
        pass
    
    def get_effective_spatial_dimension (self):
        return self.effective_spatial_dimension

    def draw (self, canvas, emitter):
        for tr in sorted (self.triangles, key = lambda tr: tr.p1.get_coords () [2]):
            tr.draw (canvas, emitter, self.coords)
    
    def rotate (self, axis, step, increment = True):
        #print ("rotating mesh")
        for tr in self.triangles:
            for p in [tr.p1, tr.p2, tr.p3]:
                ind1, ind2 = [ax for ax in axes if ax not in [axis]] [:]
                #p [ind1], p [ind2] = self._rotate_2d_vec (p [ind1], p [ind2], step)
                #print ("p", ind1, ind2)
                p.rotate_2d (ind1, ind2, step)
    
class Sphere_tri (Triangle_mesh):
    def __init__ (self, coords_, r_, color_, stripes_num_):
        Triangle_mesh.__init__ (self, coords_, r_)
        
        self.r     = r_
        self.color = color_
        
        self.stripes_num = stripes_num_
        self.generate_triangulation ()
        
    def generate_triangulation (self):
        h_step = 2 * self.r / self.stripes_num
        angle_step = 2 * math.pi / self.stripes_num
        
        for i in range (self.stripes_num):
            for j in range (self.stripes_num):
                stripe_rad_curr = math.sqrt (self.r**2 - \
                    (- self.r + i * h_step)**2)
                
                stripe_rad_next = math.sqrt (self.r**2 - \
                    (- self.r + (i + 1) * h_step)**2)
                
                #stripe_rad_curr = self.r
                #stripe_rad_next = self.r
                
                p1 = Vector ([stripe_rad_curr * math.sin (j * angle_step),
                      stripe_rad_curr * math.cos (j * angle_step),
                      - self.r + i * h_step])
                
                p2 = Vector ([stripe_rad_curr * math.sin ((j + 1) * angle_step),
                      stripe_rad_curr * math.cos ((j + 1) * angle_step),
                      - self.r + i * h_step])
                
                p3 = Vector ([stripe_rad_next * math.sin (j * angle_step),
                      stripe_rad_next * math.cos (j * angle_step),
                      - self.r + (i + 1) * h_step])
                
                p4 = Vector ([stripe_rad_next * math.sin ((j + 1) * angle_step),
                      stripe_rad_next * math.cos ((j + 1) * angle_step),
                      - self.r + (i + 1) * h_step])
                
                p5 = p2.copy ()
                p6 = p3.copy ()
                
                new_triangle_1 = Triangle (p1, p3, p2, color_ = self.color)
                new_triangle_2 = Triangle (p4, p5, p6, color_ = self.color)

                if (i > 0):
                    self.triangles.append (new_triangle_1)
                
                if (i < self.stripes_num - 1):
                    self.triangles.append (new_triangle_2)

class Canvas:
    def __init__ (self, xsz_, ysz_, zsz_, centerx_, centery_):
        self.xsz = xsz_
        self.ysz = ysz_
        self.zsz = zsz_
        self.centerx = centerx_
        self.centery = centery_
        
        self.canvas = np.ones ((WIND_Y, WIND_X, 3), np.uint8) * 55
        
    def get_canvas (self):
        return self.canvas
    
    def refresh (self):
        self.canvas = np.ones ((WIND_Y, WIND_X, 3), np.uint8) * 55
        self.draw_space_box ()
    
    def _transform_point (self, p):
        coords = p.get_coords ()
        
        x = int ((coords [0] / (coords [2] + 0) + self.centerx) * WIND_X / self.xsz)
        y = int ((coords [1] / (coords [2] + 0) + self.centery) * WIND_Y / self.ysz)
        
        return x, y
    
    def draw_3d_line (self, p1, p2, color, thickness = 1):
        x1, y1 = self._transform_point (p1)
        x2, y2 = self._transform_point (p2)
        
        cv2.line (self.canvas, (x1, y1), (x2, y2), color, thickness)
    
    def draw_3d_triangle (self, p1, p2, p3, color):
        x1, y1 = self._transform_point (p1)
        x2, y2 = self._transform_point (p2)
        x3, y3 = self._transform_point (p3)
        
        contour = np.array ([(x1, y1), (x2, y2), (x3, y3)])
        
        cv2.drawContours (self.canvas, [contour], 0, color, -1)
        
    def draw_3d_circle (self, p, r, color):
        x, y = self._transform_point (p)
        
        cv2.circle (self.canvas, (x, y), int (r / p.get_coords () [2]), color)
    
    def draw_space_box (self):
        lucc = Vector ([- self.centerx, - self.centery, 1]) #left-upper-close corner
        ludc = Vector ([- self.centerx, - self.centery, self.zsz])
        ldcc = Vector ([- self.centerx, self.ysz - self.centery, 1])
        lddc = Vector ([- self.centerx, self.ysz - self.centery, self.zsz])

        rucc = Vector ([self.xsz - self.centerx, - self.centery, 1])
        rudc = Vector ([self.xsz - self.centerx, - self.centery, self.zsz])
        rdcc = Vector ([self.xsz - self.centerx, self.ysz - self.centery, 1])
        rddc = Vector ([self.xsz - self.centerx, self.ysz - self.centery, self.zsz])
        
        self.draw_3d_line (lucc, ludc, markup_color)
        self.draw_3d_line (ldcc, lddc, markup_color)
        self.draw_3d_line (rucc, rudc, markup_color)
        self.draw_3d_line (rdcc, rddc, markup_color)

        self.draw_3d_line (ludc, rudc, markup_color)
        self.draw_3d_line (rudc, rddc, markup_color)
        self.draw_3d_line (rddc, lddc, markup_color)
        self.draw_3d_line (lddc, ludc, markup_color)
        
    def put_text (self, text, x, y, color = markup_color):
        cv2.putText (self.canvas, text, (x, y),
            cv2.FONT_HERSHEY_SIMPLEX, 1, color, 1, cv2.LINE_AA)
    
    def is_in (self, point, shift = 0):
        is_in_box = True
        
        coords = point.get_coords ()
        
        coll_axes = {"x" : 0, "y" : 0, "z" : 0}
        
        #return is_in_box, coll_axes
        
        if (coords [0] <          - self.centerx + shift or
            coords [0] > self.xsz - self.centerx - shift):
            is_in_box = False
            coll_axes ["x"] = 1

        if (coords [1] <          - self.centery + shift or
            coords [1] > self.ysz - self.centery - shift):
            is_in_box = False
            coll_axes ["y"] = 1

        if (coords [2] < 1 + shift or
            coords [2] > 1 + self.zsz - shift):
            is_in_box = False
            coll_axes ["z"] = 1
        
        return is_in_box, coll_axes

class Field:
    def __init__ (self):
        pass
    
    def calc_acceleration (self, coords):
        return Vector ([0.0, 0.0, 0.0])

class Isotrophiс_field (Field):
    def __init__ (self, acceleration_vector_):
        Field.__init__ (self)
        self.acceleration_vector = acceleration_vector_
    
    def calc_acceleration (self, coords):
        return self.acceleration_vector

class Spherically_symmetrical_field (Field):
    def __init__ (self, attraction_center_, intensity_ = 0.01):
        Field.__init__ (self)
        
        self.attraction_center = attraction_center_
        self.intensity = intensity_
    
    def calc_acceleration (self, coords):
        to_center = self.attraction_center.subtr (coords)
        distance = to_center.length ()
        
        #print (to_center.get_coords ())
        
        if (distance <= 0.3):
            distance = 0.3
        
        direction = to_center.mul (1 / distance)
        
        acceleration = direction.mul (self.intensity / distance**1)
        
        return acceleration

class Gravitating_object (Object):
    def __init__ (self, velocity_, triangle_mesh_, mass_ = 1, ignore_fields_ = False,
                  field_ = Field ()):
        Object.__init__ (self, triangle_mesh_.coords)
        
        self.velocity = velocity_
        
        self.triangle_mesh = triangle_mesh_
        self.mass = mass_
        self.ignore_fields = ignore_fields_
        self.field = field_
    
    def move (self, canvas, fields):
        effective_acceleration = Vector ([0, 0, 0])

        if (self.ignore_fields == False):
            for field in fields:
                effective_acceleration = effective_acceleration.add \
                    (field.calc_acceleration (self.coords))
        
        is_in, collision_axes = canvas.is_in (self.coords,
        self.triangle_mesh.get_effective_spatial_dimension ())

        if (is_in == False):
            for axis in collision_axes.keys ():
                if (collision_axes [axis] == 1):
                    self.velocity.change_coord (axis, invert = True)
                    self.velocity.change_coord (axis,
                        -effective_acceleration.get_coords ()
                        [axes [axis]], increment = True)
        
        self.velocity = self.velocity.add (effective_acceleration)
        self.triangle_mesh.coords = self.triangle_mesh.coords.add (self.velocity)
        self.coords = self.triangle_mesh.coords
                
    def draw (self, canvas, emitter, sun):
        self.triangle_mesh.draw (canvas, emitter)
        canvas.draw_3d_line (sun.get_coords_vec (),
                             self.triangle_mesh.get_coords_vec (),
                             self.triangle_mesh.triangles [0].get_color ())
    
    def get_effective_spatial_dimension (self):
        return self.triangle_mesh.get_effective_spatial_dimension ()
    
    def change_velocity (self, coord, val = 0, increment = False, invert = False):
        self.velocity.change_coord (coord, val, increment, invert)
    
    def get_field (self):
        return self.field

class Manager:
    def __init__ (self):
        self.emitter = Light_emitter (Vector ([0.0, 0, 0.5]))
        
        self.canvas = Canvas (2, 1.6, 3, 1, 0.8)
        #self.canvas = Canvas (4, 3.2, 6, 2, 1.6)

        self.objects = []
        
        self.to_refresh = True
        
        self.light_step = 0.2
        self.rot_step   = 0.1
        
        self.explicit_fields = []
        
    def add_object (self, obj):
        self.objects.append (obj)

    def add_objects (self, objects):
        self.objects += objects
    
    def add_explicit_field (self, field):
        self.explicit_fields.append (field)
    
    def draw (self):
        if (self.to_refresh == True):
            #self.to_refresh = False
            self.canvas.refresh ()

            #print ("rendering...")
            #before_time = time.time ()

            for obj in sorted (self.objects, key = lambda obj: -obj.get_coords () [2]):
                obj.draw (self.canvas, self.emitter, self.emitter)

            #print ("rendered in ", str (time.time () - before_time) [:6], "seconds")

            self.emitter.draw (self.canvas)
    
    def move (self):
        fields = self.explicit_fields + [obj.get_field () for obj in self.objects]
        
        for obj_ind in range (len (self.objects)):
            self.objects [obj_ind].move (self.canvas, fields)
        
#     for ind1 in range (len (self.objects)):
#         for ind2 in range (ind1 + 1, len (self.objects)):
#             obj1 = self.objects [ind1]
#             obj2 = self.objects [ind2]

#             distance = obj1.get_coords_vec ().subtr (obj2.get_coords_vec\
#                                                      ()).length ()
#             dist_th  = obj1.get_effective_spatial_dimension () + \
#                        obj1.get_effective_spatial_dimension ()

#             if (distance < dist_th):
#                 for obj in [obj1, obj2]:
#                     for axis in axes:
#                         obj.change_velocity (axis, invert = True)
        
    def get_canvas (self):
        return self.canvas.get_canvas ()
    
    def handle_keyboard (self):
        k = cv2.waitKey (1) & 0xFF
    
        if (k != 255):
            self.to_refresh = True

        if (k == ord ('q')):
            return {"exit" : True}
    
        return {"exit" : False}

#Solar system (z + 1)
#objects = [#Gravitating_object (Vector ([0, 0, 0]),
           #                    Sphere_tri (Vector ([0, 0, 0.3]), 0.2, (0, 200, 200), 14),
           #                    ignore_fields_ = True),
           #Gravitating_object (Vector ([0.04, 0.05, 0.03]),
           #                    Sphere_tri (Vector ([-0.4, -0.35, 1.6]), 0.1, (100, 200, 70), 14)),
#           Gravitating_object (Vector ([0.0, -0.06, 0.01]),
#                               Sphere_tri (Vector ([0.7, 0, 0.3]), 0.1, (100, 250, 170), 7)),
#           Gravitating_object (Vector ([0.08, -0.0, 0.01]),
#                               Sphere_tri (Vector ([0.7, 0, 0.3]), 0.12, (200, 200, 160), 6))]
#            Gravitating_object (Vector ([0.0, -0.0, 0.07]),
#                                Sphere_tri (Vector ([0.7, 0, 0.3]), 0.14, (20, 250, 150), 8)),
#            #planar
#            Gravitating_object (Vector ([0.0, -0.075, -0.01]),
#                                Sphere_tri (Vector ([0.7, 0, 0.3]), 0.15, (250, 15, 70), 7)),
#            Gravitating_object (Vector ([0.0, -0.08, -0.02]),
#                                Sphere_tri (Vector ([0.7, 0, 0.3]), 0.16, (250, 20, 170), 6)),
#            Gravitating_object (Vector ([0.0, -0.085, 0.015]),
#                                Sphere_tri (Vector ([0.7, 0, 0.3]), 0.17, (120, 200, 250), 8))
#            ]

objects = [Gravitating_object (Vector ([0.0, -0.06, 0.01]),
                               Sphere_tri (Vector ([0.7, 0, 1.2]), 0.1, (100, 250, 170), 16))]
#            Gravitating_object (Vector ([0.08, -0.0, 0.01]),
#                                Sphere_tri (Vector ([0.7, 0.5, 1.3]), 0.12, (200, 200, 250), 21)),
#            Gravitating_object (Vector ([0.0, -0.0, 0.07]),
#                                Sphere_tri (Vector ([0.7, -0.5, 1.4]), 0.14, (120, 250, 150), 16)),
#            Gravitating_object (Vector ([0.03, 0.05, 0.01]),
#                                Sphere_tri (Vector ([-0.7, 0, 1.5]), 0.13, (190, 250, 170), 20)),
#            Gravitating_object (Vector ([-0.05, -0.03, 0.01]),
#                                Sphere_tri (Vector ([-0.5, 0.5, 1.6]), 0.11, (230, 200, 160), 18)),
#            Gravitating_object (Vector ([0.03, -0.04, 0.07]),
#                                Sphere_tri (Vector ([-0.3, -0.5, 1.7]), 0.09, (220, 250, 150), 17))]
   

manager = Manager ()
manager.add_objects (objects)

manager.add_explicit_field (Isotrophiс_field (Vector ([0, 0.01, 0])))
manager.add_explicit_field (Spherically_symmetrical_field (Vector ([0, 0, 0.3])))

while (True):
    if (manager.handle_keyboard () ["exit"] == True):
        break
    
    manager.move ()
    manager.draw ()
    
    #light_info_str  = "light: " + str (emitter.get_coords ())
    #sphere_info_str = "sphere: " + str (sphere.get_coords ())
    
    #canvas.put_text (light_info_str, 20, 30)
    
    cv2.imshow ("render", manager.get_canvas ())

cv2.waitKey (0)
cv2.destroyAllWindows ()